In [1]:
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

## Cria o assistant

In [2]:
file = client.files.create(
    file=open('arquivos/divulgacao_resultado_ambev_4T23.pdf', 'rb'),
    purpose='assistants'
)

In [3]:
assistant = client.beta.assistants.create(
    name="Analista de Demonstrações Financeiras",
    instructions="Você é um analista de demonstralções \
        financeiras da Ambev. Você tem acesso a demontração \
        de resultado do 4º trimestre de 2023. Baseado apenas \
        no documento que você tem acesso, responda \
        as perguntas do usuário.",
    tools=[{'type': 'retrieval'}],
    file_ids=[file.id],
    model='gpt-4-turbo-preview'
)

## Cria uma thread

In [4]:
thread = client.beta.threads.create()

## Adiciona mensagem a thread criada

In [5]:
pergunta = 'Qual o volume de cerja vendido no Brasil segundo o documento?'

messages = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content=pergunta
)

## Roda a thread no assistant

In [6]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions='O nome do usuário é Adriano.'
)

## Aguarda a thread rodar

In [7]:
import time

while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

print(run.status)

completed


## Verifica a resposta

In [8]:
if run.status == 'completed':
    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(messages)
else:
    print('Erro', run.status)

SyncCursorPage[Message](data=[Message(id='msg_MuTgNJc9h0BrmMm39FwLpMP8', assistant_id='asst_cPJCTU3tvtgxOwnvY8vJWa0H', completed_at=None, content=[TextContentBlock(text=Text(annotations=[FileCitationAnnotation(end_index=244, file_citation=FileCitation(file_id='file-e2q6GP6IsKOSR1wfT5WkBS7K', quote="Volume ('000 hl) \nReceita líquida\nLucro bruto\n% Margem bruta\nEBITDA ajustado\n% Margem EBITDA ajustado \n\n\n4T22\n52.1693\n22.6930\n11.7078\n516% \n7.1093\n313% \n\n\n4T23\n52.0916\n19.9892\n10.6880\n535% \n7.1515\n358% \n\n\n%\nReportado\n-01% \n-119% \n-87% \n190 pb\n06% \n450 pb \n\n\n%\nOrgânico\n-01% \n295% \n364% \n280 pb\n490% \n470 pb \n\n\n12M22\n185.7497\n79.7088\n39.2868\n493% \n23.7709\n298% \n\n\n12M23\n183.6590\n79.7369\n40.4453\n507% \n25.4554\n319% \n\n\n% Reportado\n-11% \n00% \n29% \n140 pb\n71% \n210 pb \n\n\n%\nOrgânico\n-11% \n241% \n303% \n240 pb\n426% \n430 pb"), start_index=234, text='【7†source】', type='file_citation')], value='Segundo o documento, o volume de ce

In [9]:
print(messages.data[0].content[0].text.value)

Segundo o documento, o volume de cerveja vendido no Brasil pela Ambev no ano de 2023 foi de 183.659 mil hectolitros, o que representa uma redução de 11% em relação ao ano anterior, 2022, quando o volume foi de 185.7497 mil hectolitros【7†source】.


## Analisando os passos do modelo

In [10]:
run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

In [14]:
for step in run_steps.data[::-1]:
    print('======= Step >', step.step_details.type)
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            if tool_call.type == 'retrieval':
                print(tool_call)
            else:
                print('```')
                print(tool_call.code_interpreter.input)
                print('```')
                if tool_call.code_interpreter.outputs[0].type == 'logs':
                    print('Result')
                    print(tool_call.code_interpreter.outputs[0].logs)
    if step.step_details.type == 'message_creation':
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        if message.content[0].type == 'text':
            message = client.beta.threads.messages.retrieve(
                thread_id=thread.id,
                message_id=step.step_details.message_creation.message_id
            )
            print(message.content[0].text.value)

        if message.content[0].type == 'image_file':
            message = client.beta.threads.messages.retrieve(
                thread_id=thread.id,
                message_id=step.step_details.message_creation.message_id
            )
            file_id = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)

            with open(f'arquivos/{file_id}.png', 'wb') as file:
                file.write(image_data.read())

            import matplotlib.pyplot as plt
            import matplotlib.image as mpimg

            img = mpimg.imread(f'arquivos/{file_id}.png')
            fig, ax = plt.subplots()
            ax.set_axis_off()
            ax.imshow(img)
            plt.show()
        

======= Step > tool_calls
RetrievalToolCall(id='call_nMmvggvJM4iVGSEx8UFN3l05', retrieval={}, type='retrieval')
======= Step > message_creation
Segundo o documento, o volume de cerveja vendido no Brasil pela Ambev no ano de 2023 foi de 183.659 mil hectolitros, o que representa uma redução de 11% em relação ao ano anterior, 2022, quando o volume foi de 185.7497 mil hectolitros【7†source】.


In [12]:
step.step_details

ToolCallsStepDetails(tool_calls=[RetrievalToolCall(id='call_nMmvggvJM4iVGSEx8UFN3l05', retrieval={}, type='retrieval')], type='tool_calls')

In [13]:
tool_call

RetrievalToolCall(id='call_nMmvggvJM4iVGSEx8UFN3l05', retrieval={}, type='retrieval')